In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from scipy import integrate

# Setup

In [6]:
R = 1e-5        # radius of sphere
density = 2648  # density of the spheres
M = 4/3 * np.pi * R**3 * density # mass of the sphere
L = 2 * R       # distance sphere-shield
ds = 2e-9       # thickness of the shiled
epsilon_r = 4.5 # dielectric constant
dA = 100e-9     # separation of superposition
dB = dA
d = dA

# Physical constant
G = 6.67e-11    # gravitational constant
c = 3e8         # speed of light
hbar = 1.05e-34 # reduced planck constant

# Casimir = 3/2 * c/np.pi * ((epsilon_r - 1) / (epsilon_r + 2)) * R**3 * 1/(L - R - ds/2)**5
# Gravity = G*M**2 / (4 * hbar * L**2)
# DeltaPhi = G*M**2*(dA+dB)**2 / (64*hbar*L**3)

In [5]:
def t_max(L=L, dA=dA, dB=dB, alpha=np.pi/2, beta=np.pi/2):
  return 4 *hbar * np.pi * L**3 / (G * M**2 * dA * dB) * 1 / (np.sin(alpha)*np.sin(beta) - 0.5*np.cos(alpha)*np.cos(beta))

t_max_default = t_max()
print(t_max_default)

0.12863232990023374


# Numerics

In [7]:
def Casimir(t, PFA=True):
  return c * np.pi**3 / 720 * ((epsilon_r - 1) / (epsilon_r + 1)) * 0.50613 * R * t

def Gravity(t):
  return G * M**2 * t / hbar

In [8]:
def dist_A1_B1(Theta_A, Theta_B, alpha, beta):
  first_term = (2*L + d/2 * np.sin(alpha + Theta_A) - d/2 * np.sin(beta + Theta_B))
  second_term = (d/2 * np.cos(alpha + Theta_A) - d/2 * np.cos(beta + Theta_B))

  return np.sqrt(first_term**2 + second_term**2)

def dist_A2_B2(Theta_A, Theta_B, alpha, beta):
  first_term = (2*L - d/2 * np.sin(alpha + Theta_A) + d/2 * np.sin(beta + Theta_B))
  second_term = (d/2 * np.cos(alpha + Theta_A) - d/2 * np.cos(beta + Theta_B))

  return np.sqrt(first_term**2 + second_term**2)

def dist_A1_B2(Theta_A, Theta_B, alpha, beta):
  first_term = (2*L + d/2 * np.sin(alpha + Theta_A) + d/2 * np.sin(beta + Theta_B))
  second_term = (d/2 * np.cos(alpha + Theta_A) + d/2 * np.cos(beta + Theta_B))

  return np.sqrt(first_term**2 + second_term**2)

def dist_A2_B1(Theta_A, Theta_B, alpha, beta):
  first_term = (2*L - d/2 * np.sin(alpha + Theta_A) - d/2 * np.sin(beta + Theta_B))
  second_term = (d/2 * np.cos(alpha + Theta_A) + d/2 * np.cos(beta + Theta_B))

  return np.sqrt(first_term**2 + second_term**2)

In [9]:
def dist_A1(Theta_A, alpha, beta):
  return L - R - ds/2 + d/2 * np.sin(alpha + Theta_A)

def dist_A2(Theta_A, alpha, beta):
  return L - R - ds/2 - d/2 * np.sin(alpha + Theta_A)

def dist_B1(Theta_B, alpha, beta):
  return L - R - ds/2 - d/2 * np.sin(beta + Theta_B)

def dist_B2(Theta_B, alpha, beta):
  return L - R - ds/2 + d/2 * np.sin(beta + Theta_B)

In [10]:
def gaussian(Theta_A, Theta_B, Delta_Theta):
  return 1/(2 * np.pi * Delta_Theta * Delta_Theta) *  np.exp(-Theta_A**2 / (2 * Delta_Theta**2)) * np.exp(-Theta_B**2 / (2 * Delta_Theta**2))

In [11]:
def rho_12(Theta_A, Theta_B, t, alpha, beta):
  phases = Casimir(t) * (1/dist_B1(Theta_B, alpha, beta)**2 - 1/dist_B2(Theta_B, alpha, beta)**2) \
         + Gravity(t) * (1/dist_A1_B1(Theta_A, Theta_B, alpha, beta) - 1/dist_A1_B2(Theta_A, Theta_B, alpha, beta))

  return np.exp(1j * phases)

def rho_13(Theta_A, Theta_B, t, alpha, beta):
  phases = Casimir(t) * (1/dist_A1(Theta_A, alpha, beta)**2 - 1/dist_A2(Theta_A, alpha, beta)**2) \
         + Gravity(t) * (1/dist_A1_B1(Theta_A, Theta_B, alpha, beta) - 1/dist_A2_B1(Theta_A, Theta_B, alpha, beta))

  return np.exp(1j * phases)

def rho_14(Theta_A, Theta_B, t, alpha, beta):
  phases = Casimir(t) * (1/dist_A1(Theta_A, alpha, beta)**2 - 1/dist_A2(Theta_A, alpha, beta)**2) \
         + Casimir(t) * (1/dist_B1(Theta_B, alpha, beta)**2 - 1/dist_B2(Theta_B, alpha, beta)**2) \
         + Gravity(t) * (1/dist_A1_B1(Theta_A, Theta_B, alpha, beta) - 1/dist_A2_B2(Theta_A, Theta_B, alpha, beta))

  return np.exp(1j * phases)

def rho_23(Theta_A, Theta_B, t, alpha, beta):
  phases = Casimir(t) * (1/dist_A1(Theta_A, alpha, beta)**2 - 1/dist_A2(Theta_A, alpha, beta)**2) \
         + Casimir(t) * (1/dist_B2(Theta_B, alpha, beta)**2 - 1/dist_B1(Theta_B, alpha, beta)**2) \
         + Gravity(t) * (1/dist_A1_B2(Theta_A, Theta_B, alpha, beta) - 1/dist_A2_B1(Theta_A, Theta_B, alpha, beta))

  return np.exp(1j * phases)

def rho_24(Theta_A, Theta_B, t, alpha, beta):
  phases = Casimir(t) * (1/dist_A1(Theta_A, alpha, beta)**2 - 1/dist_A2(Theta_A, alpha, beta)**2) \
         + Gravity(t) * (1/dist_A1_B2(Theta_A, Theta_B, alpha, beta) - 1/dist_A2_B2(Theta_A, Theta_B, alpha, beta))

  return np.exp(1j * phases)

def rho_34(Theta_A, Theta_B, t, alpha, beta):
  phases = Casimir(t) * (1/dist_B1(Theta_B, alpha, beta)**2 - 1/dist_B2(Theta_B, alpha, beta)**2) \
         + Gravity(t) * (1/dist_A2_B1(Theta_A, Theta_B, alpha, beta) - 1/dist_A2_B2(Theta_A, Theta_B, alpha, beta))

  return np.exp(1j * phases)

In [12]:
def transposed_density(Delta_Theta, t, alpha, beta):
  limit = 8 * Delta_Theta # 1e-8

  rho12_real = integrate.dblquad(lambda Theta_A, Theta_B: np.real(rho_12(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]
  rho12_imag = integrate.dblquad(lambda Theta_A, Theta_B: np.imag(rho_12(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]

  rho13_real = integrate.dblquad(lambda Theta_A, Theta_B: np.real(rho_13(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]
  rho13_imag = integrate.dblquad(lambda Theta_A, Theta_B: np.imag(rho_13(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]

  rho14_real = integrate.dblquad(lambda Theta_A, Theta_B: np.real(rho_14(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]
  rho14_imag = integrate.dblquad(lambda Theta_A, Theta_B: np.imag(rho_14(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]

  rho23_real = integrate.dblquad(lambda Theta_A, Theta_B: np.real(rho_23(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]
  rho23_imag = integrate.dblquad(lambda Theta_A, Theta_B: np.imag(rho_23(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]

  rho24_real = integrate.dblquad(lambda Theta_A, Theta_B: np.real(rho_24(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]
  rho24_imag = integrate.dblquad(lambda Theta_A, Theta_B: np.imag(rho_24(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]

  rho34_real = integrate.dblquad(lambda Theta_A, Theta_B: np.real(rho_34(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]
  rho34_imag = integrate.dblquad(lambda Theta_A, Theta_B: np.imag(rho_34(Theta_A, Theta_B, t, alpha, beta) * gaussian(Theta_A, Theta_B, Delta_Theta)), -limit, limit, -limit, limit)[0]

  rho_T = 0.25 * np.array([
    [ 1,                           rho12_real - 1j*rho12_imag,  rho13_real + 1j*rho13_imag,  rho23_real + 1j*rho23_imag ],
    [ rho12_real + 1j*rho12_imag,  1,                           rho14_real + 1j*rho14_imag,  rho24_real + 1j*rho24_imag ],
    [ rho13_real - 1j*rho13_imag,  rho14_real - 1j*rho14_imag,  1,                           rho34_real - 1j*rho34_imag ],
    [ rho23_real - 1j*rho23_imag,  rho24_real - 1j*rho24_imag,  rho34_real + 1j*rho34_imag,  1                          ]
  ])

  return rho_T

In [13]:
def log_neg(Delta_Theta, t=t_max_default, alpha=0, beta=0):
  rho_T = transposed_density(Delta_Theta, t, alpha, beta)

  # imaginary parts are because of unstable numerics
  eigenvalues = np.real(linalg.eigvals(rho_T))
  print(eigenvalues)

  sum_negative_eigv = np.sum(np.abs(eigenvalues[eigenvalues <= 0]))
  negativity = 2 * sum_negative_eigv + 1

  log_neg = np.log(negativity) / np.log(2)

  print("Log. Neg.: ", log_neg)

  return log_neg

In [ ]:
log_neg(6e-5, t_max_default, np.pi/2, np.pi/2)

C:\Users\janbu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:1260: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
C:\Users\janbu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:1260: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
